In [1]:
from games.tic_tac_toe import TicTacToe, simple_scoring_function
from src.game.agents import AIAgent, User
from algorithms.minimax import MiniMax
from explainers.minimax_explainer import MiniMaxExplainer

# Define game

# Define tic-tac-toe scoring function for the minimax

In [2]:
explainer = MiniMaxExplainer()

In [3]:
explainer.available_frameworks()

lowlevel
highlevel


# Play and Explain

In [4]:
opponent = AIAgent(agent_id=1, core=MiniMax(simple_scoring_function, max_depth=6, use_alpha_beta=False))
user = User(agent_id=0, ask_what = False)

In [5]:
game = TicTacToe(players=[opponent, user],
                interface_mode='jupyter', 
                interface_hyperlink_mode=False)

game.act({'where' : (0,0), 'who': 0}) # First move by user

In [6]:
await game.start_game()

In [7]:
print(explainer.explain(opponent.choice, 'the best', explanation_depth=3, with_framework='highlevel'))

[['X' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']] is the context
and 
[['X' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']] O in (1, 1), id=0_3 is the best (in this context) (because
 
	[['X' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' ' ']] O in (1, 1), id=0_3 has as possible alternative moves 
	[['X' 'O' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] O in (0, 1), id=0_0, 
	[['X' ' ' 'O']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] O in (0, 2), id=0_1, 
	[['X' ' ' ' ']
	 ['O' ' ' ' ']
	 [' ' ' ' ' ']] O in (1, 0), id=0_2, 
	[['X' ' ' ' ']
	 [' ' ' ' 'O']
	 [' ' ' ' ' ']] O in (1, 2), id=0_4, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 ['O' ' ' ' ']] O in (2, 0), id=0_5, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' 'O' ' ']] O in (2, 1), id=0_6, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'O']] O in (2, 2), id=0_7
	and 
	[['X' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' ' ']] O in (1, 1), id=0_3 is better than them (because
	 	it leads to a better position (because
 
			[['X' ' ' ' ']
			 [' ' 'O' ' ']
			 [' ' ' ' ' ']] O in (1, 1), id=0_3 has a

In [8]:
print(explainer.explain(opponent.choice, 'best', explanation_depth=4, with_framework='lowlevel'))

[['X' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']] is the context
and 
[['X' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']] O in (1, 1), id=0_3 is best (in this context) (because
 
	[['X' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' ' ']] O in (1, 1), id=0_3 has siblings 
	[['X' 'O' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] O in (0, 1), id=0_0, 
	[['X' ' ' 'O']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] O in (0, 2), id=0_1, 
	[['X' ' ' ' ']
	 ['O' ' ' ' ']
	 [' ' ' ' ' ']] O in (1, 0), id=0_2, 
	[['X' ' ' ' ']
	 [' ' ' ' 'O']
	 [' ' ' ' ' ']] O in (1, 2), id=0_4, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 ['O' ' ' ' ']] O in (2, 0), id=0_5, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' 'O' ' ']] O in (2, 1), id=0_6, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'O']] O in (2, 2), id=0_7
	and 
	[['X' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' ' ']] O in (1, 1), id=0_3 is better than them (because
 
			[['X' ' ' ' ']
			 [' ' 'O' ' ']
			 [' ' ' ' ' ']] O in (1, 1), id=0_3 has score 0.0 (because
 
						[['X' ' ' ' ']
						 [' ' 'O' ' ']
					

In [9]:
#explainer.del_explanation_tactic('OnlyRelevantComparisons', to_framework='highlevel', to_adjective='the best') this tactic needs fixing

In [10]:
print(explainer.explain(opponent.choice, 'best', explanation_depth=2))

[['X' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']] is the context
and 
[['X' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']] O in (1, 1), id=0_3 is best (in this context) (because
 
	[['X' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' ' ']] O in (1, 1), id=0_3 has siblings 
	[['X' 'O' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] O in (0, 1), id=0_0, 
	[['X' ' ' 'O']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] O in (0, 2), id=0_1, 
	[['X' ' ' ' ']
	 ['O' ' ' ' ']
	 [' ' ' ' ' ']] O in (1, 0), id=0_2, 
	[['X' ' ' ' ']
	 [' ' ' ' 'O']
	 [' ' ' ' ' ']] O in (1, 2), id=0_4, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 ['O' ' ' ' ']] O in (2, 0), id=0_5, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' 'O' ' ']] O in (2, 1), id=0_6, 
	[['X' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'O']] O in (2, 2), id=0_7
	and 
	[['X' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' ' ']] O in (1, 1), id=0_3 is better than them (because
 
			[['X' ' ' ' ']
			 [' ' 'O' ' ']
			 [' ' ' ' ' ']] O in (1, 1), id=0_3 has score 0.0
			and 
			[['X' 'O' ' ']
			 [' ' ' ' ' ']
			 [' ' ' ' '